In [14]:
from pyspark.sql import SparkSession

In [19]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Task_3") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [16]:
df_path = "/home/rupeshr/Desktop/TSA_Python/dataset/time series dataset/archive/Electric_Production1.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show(30)

root
 |-- _c0: integer (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- IPG2211A2N: double (nullable = true)

+---+-------------------+----------+
|_c0|               DATE|IPG2211A2N|
+---+-------------------+----------+
|  0|1985-01-01 00:00:00|   72.5052|
|  1|1985-02-01 00:00:00|    70.672|
|  2|1985-03-01 00:00:00|   62.4502|
|  3|1985-04-01 00:00:00|   57.4714|
|  4|1985-05-01 00:00:00|   55.3151|
|  5|1985-06-01 00:00:00|   58.0904|
|  6|1985-07-01 00:00:00|   62.6202|
|  7|1985-08-01 00:00:00|   63.2485|
|  8|1985-09-01 00:00:00|   60.5846|
|  9|1985-10-01 00:00:00|   56.3154|
| 10|1985-11-01 00:00:00|   58.0005|
| 11|1985-12-01 00:00:00|   68.7145|
| 12|1986-01-01 00:00:00|   73.3057|
| 13|1986-02-01 00:00:00|   67.9869|
| 14|1986-03-01 00:00:00|   62.2221|
| 15|1986-04-01 00:00:00|   57.0329|
| 16|1986-05-01 00:00:00|   55.8137|
| 17|1986-06-01 00:00:00|   59.9005|
| 18|1986-07-01 00:00:00|   65.7655|
| 19|1986-08-01 00:00:00|   64.4816|
| 20|1986-09-01 00:00:00| 

In [17]:
def extreme_date(df,datecolumn):
    start=df.select(min(datecolumn)).collect()
    end=df.select(max(datecolumn)).collect()
    x={'start':start, 'end':end}
    return x
x=extreme_date(df,'DATE')
print(x)

{'start': [Row(min(DATE)=datetime.datetime(1985, 1, 1, 0, 0))], 'end': [Row(max(DATE)=datetime.datetime(2018, 1, 1, 0, 0))]}


In [18]:
def filter_dataslicing(df,datecolumn,start,end):
    df=df.filter((df[datecolumn] >= start) & (df[datecolumn] <= end))
    return df
df1=filter_dataslicing(df,datecolumn='DATE',start='1986-01-01 00:00:00',end='1986-10-01 00:00:00')
df1.show()

+---+-------------------+----------+
|_c0|               DATE|IPG2211A2N|
+---+-------------------+----------+
| 12|1986-01-01 00:00:00|   73.3057|
| 13|1986-02-01 00:00:00|   67.9869|
| 14|1986-03-01 00:00:00|   62.2221|
| 15|1986-04-01 00:00:00|   57.0329|
| 16|1986-05-01 00:00:00|   55.8137|
| 17|1986-06-01 00:00:00|   59.9005|
| 18|1986-07-01 00:00:00|   65.7655|
| 19|1986-08-01 00:00:00|   64.4816|
| 20|1986-09-01 00:00:00|   61.0005|
| 21|1986-10-01 00:00:00|   57.5322|
+---+-------------------+----------+



In [11]:
data=df.toPandas()
data.head()

,_c0,DATE,IPG2211A2N
0,0,1985-01-01,72.5052
1,1,1985-02-01,70.6720
2,2,1985-03-01,62.4502
3,3,1985-04-01,57.4714
4,4,1985-05-01,55.3151


In [12]:
data = spark.createDataFrame(data)
data.show()

+---+-------------------+----------+
|_c0|               DATE|IPG2211A2N|
+---+-------------------+----------+
|  0|1985-01-01 00:00:00|   72.5052|
|  1|1985-02-01 00:00:00|    70.672|
|  2|1985-03-01 00:00:00|   62.4502|
|  3|1985-04-01 00:00:00|   57.4714|
|  4|1985-05-01 00:00:00|   55.3151|
|  5|1985-06-01 00:00:00|   58.0904|
|  6|1985-07-01 00:00:00|   62.6202|
|  7|1985-08-01 00:00:00|   63.2485|
|  8|1985-09-01 00:00:00|   60.5846|
|  9|1985-10-01 00:00:00|   56.3154|
| 10|1985-11-01 00:00:00|   58.0005|
| 11|1985-12-01 00:00:00|   68.7145|
| 12|1986-01-01 00:00:00|   73.3057|
| 13|1986-02-01 00:00:00|   67.9869|
| 14|1986-03-01 00:00:00|   62.2221|
| 15|1986-04-01 00:00:00|   57.0329|
| 16|1986-05-01 00:00:00|   55.8137|
| 17|1986-06-01 00:00:00|   59.9005|
| 18|1986-07-01 00:00:00|   65.7655|
| 19|1986-08-01 00:00:00|   64.4816|
+---+-------------------+----------+
only showing top 20 rows



In [ ]:
def extreme_date(df,datecolumn):
    start=df.select(min(datecolumn)).collect()
    end=df.select(max(datecolumn)).collect()
    x={'start':start, 'end':end}
    return x

class timeseries_extreme_date(APIView):
    def post(self,request):
        #auth_keyword, token = get_authorization_header(request).split()
        #token = token.decode("utf-8")
        #check = headercheck(token)
        check ='ok'
        if check == 'ok':
            pass
        else:
            return Response({"status": "validation", "error": "user has been killed"})
        data = request.data
        path=data['path']
        date=data['date']
        try:
            import numpy as np
            #file = match_version(version, analysis, filename)
            #data = read_file(file)  # spark data frame
            #if str(type(data)) == "<class 'str'>":
             #   data = read_file(filename)
            #data = data.toPandas()
            df=pd.read_csv(path)
            df[date] = pd.to_datetime(df[date])
            df = df.sort_values(by=date)
            df = spark.createDataFrame(df)
            #df.set_index(date, inplace=True)
            output=extreme_date(df,date)
            print(output)
            return Response({'status':'success'})
        except Exception as e:
            traceback.print_exc()
            return Response({"status": "failure","error":"Please refer the logs in Logs->Con

In [ ]:
def filter_dataslicing(df,datecolumn,start,end):
    df=df.filter((df[datecolumn] >= start) & (df[datecolumn] <= end))
    return df

class timeseries_dataslicing(APIView):
    def post(self,request):
        #auth_keyword, token = get_authorization_header(request).split()
        #token = token.decode("utf-8")
        #check = headercheck(token)
        check ='ok'
        if check == 'ok':
            pass
        else:
            return Response({"status": "validation", "error": "user has been killed"})
        data = request.data
        path=data['path']
        date=data['date']
        start=data['startdate']
        end=data['enddate']
        try:
            import numpy as np
            #file = match_version(version, analysis, filename)
            #data = read_file(file)  # spark data frame
            #if str(type(data)) == "<class 'str'>":
             #   data = read_file(filename)
            #data = data.toPandas()
            df=pd.read_csv(path)
            df[date] = pd.to_datetime(df[date])
            df = df.sort_values(by=date)
            df = spark.createDataFrame(df)
            df=filter_dataslicing(df,date,start,end)
            print(df)
            return Response({'status':'success'})
        except Exception as e:
            traceback.print_exc()
            return Response({"status": "failure","error":"Please refer the logs in Logs->Container Log"})